In [37]:
import geemap
import ee
import ipywidgets as widgets
from ipyleaflet import WidgetControl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
import numpy as np
import os

In [38]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [39]:
# From
date01_wgt = widgets.DatePicker(
    description='From',
    value = dt.date(2021, 1, 1),
    layout=widgets.Layout(width="215px")
)
# To
date02_wgt = widgets.DatePicker(
    description='To',
    value = dt.date(2022, 1,1),
    layout=widgets.Layout(width="215px")
)

In [40]:
date01_wgt

DatePicker(value=datetime.date(2021, 1, 1), description='From', layout=Layout(width='215px'))

In [41]:
date02_wgt

DatePicker(value=datetime.date(2022, 1, 1), description='To', layout=Layout(width='215px'))

In [42]:
def timer1():    
    ss = date01_wgt.value
    date01 = "{}-{:02}-{:02}".format(ss.year, ss.month, ss.day)
    return(date01)
    
def timer2():
    ss = date02_wgt.value
    date02 = "{}-{:02}-{:02}".format(ss.year, ss.month, ss.day)
    return(date02)
    
    

In [13]:
#button_conf = widgets.Button(description='Ok', button_style='primary', tooltip='Click me', layout=widgets.Layout(width="40px"))
output_2 = widgets.Output()

layer = widgets.Dropdown(
    options=["Aerosol", "Cloud", "NO2", "CO", "CH2O", 'O3', 'SO2', "CH4"],
    value=None,
    description="Layer:",
    #layout=widgets.Layout(width=widget_width, padding=padding),
    style={"description_width": "initial"},
)
widgets.HBox([layer])

In [14]:
band_wdg = widgets.Dropdown(
            options=["comino", "ajo"],
            value=None,
            description="Band:",
            #layout=widgets.Layout(width=widget_width, padding=padding),
            style={"description_width": "initial"},
        )
prubox = widgets.VBox([band_wdg, output_2])
prubox

In [15]:
def handle_dropdwn_click(change):
    
    if change['new'] == 'Aerosol':
        band_wdg.options = ["absorbing_aerosol_index",
                            "sensor_altitude", 
                            "sensor_azimuth_angle",
                            "sensor_zenith_angle",
                            "solar_azimuth_angle",
                            "solar_zenith_angle"]
    
    elif change['new'] == 'Cloud':
        band_wdg.options = ["cloud_fraction",
                            "cloud_top_pressure",
                            "cloud_top_height",
                            "cloud_base_pressure",
                            "cloud_base_height",
                            "cloud_optical_depth",
                            "surface_albedo",
                            "sensor_azimuth_angle",
                            "sensor_zenith_angle",
                            "solar_azimuth_angle",
                            "solar_zenith_angle"]
        
    elif change['new'] == 'NO2':
        band_wdg.options = ["NO2_column_number_density",
                            "tropospheric_NO2_column_number_density",
                            "stratospheric_NO2_column_number_density",
                            "NO2_slant_column_number_density",
                            "tropopause_pressure,"
                            "absorbing_aerosol_index",
                            "cloud_fraction",
                            "sensor_altitude",
                            "sensor_azimuth_angle",
                            "sensor_zenith_angle",
                            "solar_azimuth_angle",
                            "solar_zenith_angle"]
        
    elif change['new'] == 'CO':
        band_wdg.options = ["CO_column_number_density",
                            "H2O_column_number_density",
                            "cloud_height",
                            "sensor_altitude",
                            "sensor_azimuth_angle",
                            "sensor_zenith_angle",
                            "solar_azimuth_angle",
                            "solar_zenith_angle"]
        
    elif change['new'] == 'CH2O':
        band_wdg.options = ["tropospheric_HCHO_column_number_density",
                            "tropospheric_HCHO_column_number_density_amf",
                            "HCHO_slant_column_number_density",
                            "cloud_fraction",
                            "sensor_azimuth_angle",
                            "sensor_zenith_angle",
                            "solar_azimuth_angle",
                            "solar_zenith_angle"]
        
    elif change['new'] == 'O3':
        band_wdg.options = ["O3_column_number_density",
                            "O3_effective_temperature",
                            "cloud_fraction",
                            "sensor_azimuth_angle",
                            "sensor_zenith_angle",
                            "solar_azimuth_angle",
                            "solar_zenith_angle"]
        
    elif change['new'] == 'SO2':
        band_wdg.options = ["SO2_column_number_density",
                            "SO2_column_number_density_amf",
                            "SO2_slant_column_number_density",
                            "absorbing_aerosol_index",
                            "cloud_fraction",
                            "sensor_azimuth_angle",
                            "sensor_zenith_angle",
                            "solar_azimuth_angle",
                            "solar_zenith_angle",
                            "SO2_column_number_density_15km"]
        
    elif change['new'] == 'CH4':
        band_wdg.options = ["CH4_column_volume_mixing_ratio_dry_air",
                            "aerosol_height",
                            "aerosol_optical_depth",
                            "sensor_azimuth_angle",
                            "sensor_zenith_angle",
                            "solar_azimuth_angle",
                            "solar_zenith_angle",
                            "CH4_column_volume_mixing_ratio_dry_air_bias_corrected",
                            "CH4_column_volume_mixing_ratio_dry_air_uncertainty"]

        
layer.observe(handle_dropdwn_click, "value")

In [17]:
button = widgets.Button(description='Draw', button_style='primary', tooltip='Click me', ayout=widgets.Layout(width="100px"))

output = widgets.Output()

def clicking(b):
    with output:
        if layer.value == "Aerosol":
            collection_aerosol = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_AER_AI') \
  .select('absorbing_aerosol_index') \
  .filterDate(timer1(), timer2())

            band_viz = {
  'min': -1,
  'max': 2.0,
  'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}
            Map.addLayer(collection_aerosol.mean(), band_viz, 'S5P Aerosol')
        
        if layer.value == "Cloud":
            collection_cloud = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_CLOUD') \
  .select('cloud_fraction') \
  .filterDate(timer1(), timer2())

            band_viz = {
  'min': 0,
  'max': 0.95,
  'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

            Map.addLayer(collection_cloud.mean(), band_viz, 'S5P Cloud')
        
        if layer.value == "NO2":
            collection_no2 = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_NO2') \
  .select('tropospheric_NO2_column_number_density') \
  .filterDate(timer1(), timer2())

            band_viz = {
  'min': 0,
  'max': 0.0002,
  'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

            Map.addLayer(collection_no2.mean(), band_viz, 'S5P N02')
        
        if layer.value == "O3":
            collection_o3 = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_O3') \
  .select('O3_column_number_density') \
  .filterDate(timer1(), timer2())

            band_viz = {
  'min': 0.12,
  'max': 0.15,
  'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

            Map.addLayer(collection_o3.mean(), band_viz, 'S5P O3')
            
        if layer.value == "SO2":
            collection_so2 = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_SO2') \
  .select('SO2_column_number_density') \
  .filterDate(timer1(), timer2())

            band_viz = {
  'min': 0.0,
  'max': 0.0005,
  'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

            Map.addLayer(collection_so2.mean(), band_viz, 'S5P SO2')
        
        if layer.value == "CH4":
            collection_ch4 = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_CH4') \
  .select('CH4_column_volume_mixing_ratio_dry_air') \
  .filterDate(timer1(), timer2())

            band_viz = {
  'min': 1750,
  'max': 1900,
  'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

            Map.addLayer(collection_ch4.mean(), band_viz, 'S5P CH4')
        
        if layer.value == "CO":
            collection_co = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_CO') \
  .select('CO_column_number_density') \
  .filterDate(timer1(), timer2())

            band_viz = {
  'min': 0,
  'max': 0.05,
  'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

            Map.addLayer(collection_co.mean(), band_viz, 'S5P CO')
        
        if layer.value == "CH2O":
            collection_ch2o = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_HCHO') \
  .select('tropospheric_HCHO_column_number_density') \
  .filterDate(timer1(), timer2())

            band_viz = {
  'min': 0.0,
  'max': 0.0003,
  'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

            Map.addLayer(collection_ch2o.mean(), band_viz, 'S5P HCHO')
        
        print(timer1(), timer2())
            
button.on_click(clicking)

widgets.VBox([button, output])

In [45]:
print(timer1(), timer2())

2021-01-01 2022-01-01
